# CS6140 Assignments

**Instructions**
1. In each assignment cell, look for the block:
 ```
  #BEGIN YOUR CODE
  raise NotImplementedError.new()
  #END YOUR CODE
 ```
1. Replace this block with your solution.
1. Test your solution by running the cells following your block (indicated by ##TEST##)
1. Click the "Validate" button above to validate the work.

**Notes**
* You may add other cells and functions as needed
* Keep all code in the same notebook
* In order to receive credit, code must "Validate" on the JupyterHub server

---

# Final Project: Part 1 - Exploratory Analysis


We are going to do some exploratory analysis on the data to see what is available. This data is from a [Kaggle competition](https://www.kaggle.com/c/home-credit-default-risk/data). Although we are not going to be entering the competition, we will come up with something reasonable. See the associated Final Project note in the course OneNote on the acceptance criteria and requirements. 

The final project consists of three SQLite databases. The databases below are identical but sampled as needed. Note that databases are read-only so you should not try to create or alter the contents. You may create another database in your directory.

* Train: Used for full model training, as needed.
* Dev: Used for quick prototyping, validation, etc.
* Test: Used to grade and evaluate your work (Note: Used when grading only).

They all have the same schema, shown below. Note that there is no "application_test". 

![alt text](home_credit.png)


In [43]:
require './assignment_lib.rb'

dir = "/home/dataset"
$dev_db = SQLite3::Database.new "#{dir}/credit_risk_data_dev.db", results_as_hash: true, readonly: true
$train_db = SQLite3::Database.new "#{dir}/credit_risk_data_train.db", results_as_hash: true, readonly: true

#<SQLite3::Database:0x0000000002684f50 @tracefunc=nil, @authorizer=nil, @encoding=nil, @busy_handler=nil, @collations={}, @functions={}, @results_as_hash=true, @type_translation=nil, @readonly=true>

## Getting to know the data
Here are some sample queries to show the tables and columns for a table. You are encourages to play around with the data and export some data to your favorite data analysis system.

In [44]:
sql = <<SQL
SELECT name FROM sqlite_master
            WHERE type='table'
            ORDER BY name;"
SQL

$dev_db.execute(sql) do |row|
  puts row
end

{"name"=>"POS_CASH_balance", 0=>"POS_CASH_balance"}
{"name"=>"application_train", 0=>"application_train"}
{"name"=>"bureau", 0=>"bureau"}
{"name"=>"bureau_balance", 0=>"bureau_balance"}
{"name"=>"credit_card_balance", 0=>"credit_card_balance"}
{"name"=>"data_sample", 0=>"data_sample"}
{"name"=>"installments_payments", 0=>"installments_payments"}
{"name"=>"previous_application", 0=>"previous_application"}


#<SQLite3::Statement:0x000000000261dda0 @connection=#<SQLite3::Database:0x0000000002685090 @tracefunc=nil, @authorizer=nil, @encoding=#<Encoding:UTF-8>, @busy_handler=nil, @collations={}, @functions={}, @results_as_hash=true, @type_translation=nil, @readonly=true>, @remainder="\"\n", @columns=["name"], @types=["text"]>

In [45]:
sql = "select * from application_train"
$dev_db.prepare(sql).columns

["SK_ID_CURR", "TARGET", "NAME_CONTRACT_TYPE", "CODE_GENDER", "FLAG_OWN_CAR", "FLAG_OWN_REALTY", "CNT_CHILDREN", "AMT_INCOME_TOTAL", "AMT_CREDIT", "AMT_ANNUITY", "AMT_GOODS_PRICE", "NAME_TYPE_SUITE", "NAME_INCOME_TYPE", "NAME_EDUCATION_TYPE", "NAME_FAMILY_STATUS", "NAME_HOUSING_TYPE", "REGION_POPULATION_RELATIVE", "DAYS_BIRTH", "DAYS_EMPLOYED", "DAYS_REGISTRATION", "DAYS_ID_PUBLISH", "OWN_CAR_AGE", "FLAG_MOBIL", "FLAG_EMP_PHONE", "FLAG_WORK_PHONE", "FLAG_CONT_MOBILE", "FLAG_PHONE", "FLAG_EMAIL", "OCCUPATION_TYPE", "CNT_FAM_MEMBERS", "REGION_RATING_CLIENT", "REGION_RATING_CLIENT_W_CITY", "WEEKDAY_APPR_PROCESS_START", "HOUR_APPR_PROCESS_START", "REG_REGION_NOT_LIVE_REGION", "REG_REGION_NOT_WORK_REGION", "LIVE_REGION_NOT_WORK_REGION", "REG_CITY_NOT_LIVE_CITY", "REG_CITY_NOT_WORK_CITY", "LIVE_CITY_NOT_WORK_CITY", "ORGANIZATION_TYPE", "EXT_SOURCE_1", "EXT_SOURCE_2", "EXT_SOURCE_3", "APARTMENTS_AVG", "BASEMENTAREA_AVG", "YEARS_BEGINEXPLUATATION_AVG", "YEARS_BUILD_AVG", "COMMONAREA_AVG", "ELE

## Question 1.1 (20 points)

Getting familiar with SQL, try counting the number of rows in the dev database and the train database. Return a single number from an SQL query.

Here is an example of how to run a query:

```ruby
def count_applications db
    db.execute("select * from application_train limit 10") do |row|
        puts row["TARGET"]
    end
end
```

In [46]:
def count_applications db
  # BEGIN YOUR CODE
  count = 0

  sql = "select count(*) from application_train"
  
  db.execute(sql) do |row|
    puts row
    count = row["count(*)"]
  end
  count
  #END YOUR CODE
end

:count_applications

In [47]:
def test_11()
  assert_equal 15334, count_applications($dev_db)
  assert_equal 276667, count_applications($train_db)
end

test_11()

{"count(*)"=>15334, 0=>15334}
{"count(*)"=>276667, 0=>276667}


## Begin Question 1.2 (10 points)

This dataset presents a classification problem in which we seek to predict positive (target = 1) applications versus negative (target = 0) applications. Each application is identified by ```SK_ID_CURR``` in a record of the table ```application_train```. Other tables join on the ```SK_ID_CURR``` field. 

Return the number of applications with target value 0 and 1 as a hash. Name the counts as "n". 

In [48]:
def target_distribution(db)
  # BEGIN YOUR CODE
  distribution = Hash.new
  sql_0 = "select count(*) 
        from application_train
        where target = 0"
  sql_1 = "select count(*) 
        from application_train 
        where target = 1"

  db.execute(sql_0) do |row|
    count_0 = row["count(*)"]
    distribution[0] = count_0
  end
  
  db.execute(sql_1) do |row|
    count_1 = row["count(*)"]
    distribution[1] = count_1
  end
  
  #END YOUR CODE
  puts distribution
  return distribution
  
end

:target_distribution

In [49]:
# Tests for the counts
def test_12()
  dist = target_distribution($dev_db)
  assert_equal 14101, dist[0]
  assert_equal 1233, dist[1]
end
test_12()

{0=>14101, 1=>1233}


## Begin Question 1.3 (10 points)

Write a function which, when provided a database and a name of a column with categorical values, returns a hash with the top 10 values, along with their counts. 


In [50]:
def feature_frequency db, fname
  frequency = Hash.new
  # BEGIN YOUR CODE
  sql = "select " + fname.to_s + " 
         from application_train"
  
  i = 0
  freq_map = Hash.new
  db.execute(sql) do |row|
    key = row[fname]
    if freq_map.key?(key)
      freq_map[key] += 1
    else
      freq_map[key] = 1
    end
  end
  freq_entry = freq_map.to_a.sort_by {|row| row[1]}
  freq_keys = freq_entry.last(10).map{|e| e[0]}
  freq_keys.reverse.each do |key|
    frequency[key] = freq_map[key]
  end
   
  #END YOUR CODE
  return frequency
end


:feature_frequency

In [51]:
def test_cos
  d_ct = feature_frequency $dev_db, "FLAG_OWN_REALTY"
  assert_equal 2, d_ct.size
  assert_equal 10618, d_ct["Y"]
  assert_equal 4716, d_ct["N"]
end

test_cos()

In [52]:
def test_13
  d_ct = feature_frequency $dev_db, "NAME_CONTRACT_TYPE"
  assert_equal 2, d_ct.size
  assert_equal 13867, d_ct["Cash loans"]
  assert_equal 1467, d_ct["Revolving loans"]
end

test_13()

## Begin Question 1.4 (10 points)

Count the number of positive (target = 1) and negative (target = 0) applications from ```application_train``` with exactly 4 records in ```previous_application```.

In [53]:
def num_with_prev(db)
  counts = Hash.new
  # BEGIN YOUR CODE
  freq_map = Hash.new
  
  sql_pre = "select SK_ID_CURR 
         from previous_application 
         group by SK_ID_CURR 
         having count(*) == 4"
  sql = "select target, count(*)
         from application_train 
         where SK_ID_CURR in (" + sql_pre + ") 
         group by target"
  db.execute(sql) do |row|
    puts row
    target = row[0]
    counts[target] = row[1]
  end
  puts counts
  
  
#   #END YOUR CODE
  return counts
end

:num_with_prev

In [54]:
def test_14()
  t = num_with_prev($dev_db)
  assert_true(t.has_key? 0)
  assert_true(t.has_key? 1)
end

test_14()

{"TARGET"=>0, "count(*)"=>1514, 0=>0, 1=>1514}
{"TARGET"=>1, "count(*)"=>124, 0=>1, 1=>124}
{0=>1514, 1=>124}


In [55]:
def test_cos()
  t = num_with_prev($dev_db)
  assert_equal 1514, t[0]
  assert_equal 124, t[1]
end

test_cos()

{"TARGET"=>0, "count(*)"=>1514, 0=>0, 1=>1514}
{"TARGET"=>1, "count(*)"=>124, 0=>1, 1=>124}
{0=>1514, 1=>124}
